In [42]:
import os
import shutil
import pandas as pd
import subprocess
from pathlib import Path

In [10]:
df = pd.read_csv('../data/casp14_stage2.csv', index_col=0)

In [11]:
df

,3DCNN_prof,ProQ3D,GraphQA,angleQA,ProQ2,DAVIS-EMAconsensus,BAKER-ROSETTASERVER,BAKER-experimental,P3De,ModFOLD8,...,MolPrb_ramfv,N1,N2,DIST,N,RMSD,GDT_TS,LGA_S3,LGA_Q,Target
T1024TS075_1,0.66066,0.586251,0.649,0.467,0.598,0.64812,0.6782,0.6175,0.567581,0.581992,...,95.32,408.0,391.0,4.0,203.0,2.48,47.059,45.012,7.868,T1024
T1024TS050_4,0.56958,0.540781,0.579,0.416,0.547,0.62409,0.6012,0.5421,0.532019,0.570982,...,93.10,408.0,391.0,4.0,224.0,2.69,50.256,48.095,8.024,T1024
T1024TS211_4,0.56007,0.531768,0.584,0.419,0.541,0.61050,0.6253,0.5918,0.492319,0.536132,...,84.68,374.0,391.0,4.0,199.0,2.32,46.036,46.466,8.220,T1024
T1024TS367_5,0.69797,0.684465,0.646,0.601,0.610,0.68982,0.7432,0.7308,0.613875,0.578979,...,96.80,408.0,391.0,4.0,328.0,2.33,60.934,62.641,13.478,T1024
T1024TS278_1,0.69687,0.750612,0.716,0.487,0.738,0.68287,0.7267,0.6677,0.702875,0.557215,...,97.54,408.0,391.0,4.0,289.0,2.26,59.910,55.603,12.245,T1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1101TS013_5,0.67675,0.653284,0.719,0.708,0.791,0.51803,0.7385,0.7623,0.625477,0.527876,...,94.30,318.0,297.0,4.0,172.0,1.94,47.138,43.689,8.418,T1101
T1101TS031_4,0.71093,0.602920,0.668,0.693,0.666,0.55879,0.6791,0.6607,0.594282,0.537432,...,90.19,318.0,297.0,4.0,188.0,1.88,51.010,48.073,9.512,T1101
T1101TS140_5,0.68995,0.624382,0.704,0.762,0.676,0.57418,0.7006,0.7311,0.590116,0.562642,...,91.14,318.0,297.0,4.0,200.0,2.56,57.239,50.569,7.505,T1101
T1101TS435_4,0.66884,0.533206,0.666,0.692,0.691,0.54556,0.6293,0.6036,0.549051,0.506801,...,79.43,318.0,297.0,4.0,168.0,2.40,49.579,43.283,6.709,T1101


In [31]:
drop_df = df.dropna(subset=['GDT_TS'])
drop_df = drop_df.groupby('Target').filter(lambda x: x['GDT_TS'].max() > 40)
fil_target = ['T1027', 'T1088'] + ['T1048', 'T1062', 'T1072s1', 'T1098']
fil_df = drop_df.groupby('Target').filter(lambda x: x.name not in fil_target)
fil_df

,3DCNN_prof,ProQ3D,GraphQA,angleQA,ProQ2,DAVIS-EMAconsensus,BAKER-ROSETTASERVER,BAKER-experimental,P3De,ModFOLD8,...,MolPrb_ramfv,N1,N2,DIST,N,RMSD,GDT_TS,LGA_S3,LGA_Q,Target
T1024TS075_1,0.66066,0.586251,0.649,0.467,0.598,0.64812,0.6782,0.6175,0.567581,0.581992,...,95.32,408.0,391.0,4.0,203.0,2.48,47.059,45.012,7.868,T1024
T1024TS050_4,0.56958,0.540781,0.579,0.416,0.547,0.62409,0.6012,0.5421,0.532019,0.570982,...,93.10,408.0,391.0,4.0,224.0,2.69,50.256,48.095,8.024,T1024
T1024TS211_4,0.56007,0.531768,0.584,0.419,0.541,0.61050,0.6253,0.5918,0.492319,0.536132,...,84.68,374.0,391.0,4.0,199.0,2.32,46.036,46.466,8.220,T1024
T1024TS367_5,0.69797,0.684465,0.646,0.601,0.610,0.68982,0.7432,0.7308,0.613875,0.578979,...,96.80,408.0,391.0,4.0,328.0,2.33,60.934,62.641,13.478,T1024
T1024TS278_1,0.69687,0.750612,0.716,0.487,0.738,0.68287,0.7267,0.6677,0.702875,0.557215,...,97.54,408.0,391.0,4.0,289.0,2.26,59.910,55.603,12.245,T1024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T1101TS013_5,0.67675,0.653284,0.719,0.708,0.791,0.51803,0.7385,0.7623,0.625477,0.527876,...,94.30,318.0,297.0,4.0,172.0,1.94,47.138,43.689,8.418,T1101
T1101TS031_4,0.71093,0.602920,0.668,0.693,0.666,0.55879,0.6791,0.6607,0.594282,0.537432,...,90.19,318.0,297.0,4.0,188.0,1.88,51.010,48.073,9.512,T1101
T1101TS140_5,0.68995,0.624382,0.704,0.762,0.676,0.57418,0.7006,0.7311,0.590116,0.562642,...,91.14,318.0,297.0,4.0,200.0,2.56,57.239,50.569,7.505,T1101
T1101TS435_4,0.66884,0.533206,0.666,0.692,0.691,0.54556,0.6293,0.6036,0.549051,0.506801,...,79.43,318.0,297.0,4.0,168.0,2.40,49.579,43.283,6.709,T1101


In [32]:
len(fil_df.groupby('Target'))

58

In [36]:
import shutil

for target, group in df.groupby('Target'):
    target_outdir = Path('../../pdb/stage_2/') / target
    target_outdir.mkdir(parents=True, exist_ok=True)
    print(target_outdir)
    target_indir = Path('../predictions/regular') / target
    for index in group.index:
        outpath = target_outdir / index
        if not outpath.exists():
            shutil.copy(target_indir / index, outpath)

../../pdb/stage_2/T1024
../../pdb/stage_2/T1025
../../pdb/stage_2/T1026
../../pdb/stage_2/T1027
../../pdb/stage_2/T1028
../../pdb/stage_2/T1029
../../pdb/stage_2/T1030
../../pdb/stage_2/T1031
../../pdb/stage_2/T1032
../../pdb/stage_2/T1033
../../pdb/stage_2/T1034
../../pdb/stage_2/T1035
../../pdb/stage_2/T1036s1
../../pdb/stage_2/T1037
../../pdb/stage_2/T1038
../../pdb/stage_2/T1039
../../pdb/stage_2/T1040
../../pdb/stage_2/T1041
../../pdb/stage_2/T1042
../../pdb/stage_2/T1043
../../pdb/stage_2/T1045s1
../../pdb/stage_2/T1045s2
../../pdb/stage_2/T1046s1
../../pdb/stage_2/T1046s2
../../pdb/stage_2/T1047s1
../../pdb/stage_2/T1047s2
../../pdb/stage_2/T1048
../../pdb/stage_2/T1049
../../pdb/stage_2/T1050
../../pdb/stage_2/T1052
../../pdb/stage_2/T1053
../../pdb/stage_2/T1054
../../pdb/stage_2/T1055
../../pdb/stage_2/T1056
../../pdb/stage_2/T1057
../../pdb/stage_2/T1058
../../pdb/stage_2/T1060s2
../../pdb/stage_2/T1060s3
../../pdb/stage_2/T1061
../../pdb/stage_2/T1062
../../pdb/stage_2/T106

## cp alphafold model

In [43]:
for target, group in df.groupby('Target'):
    target_outdir = Path('../../pdb/alphafold2/') / target
    target_outdir.mkdir(parents=True, exist_ok=True)
    print(target_outdir)
    target_indir = Path('../predictions/regular') / target
    for i in range(1, 6):
        file_name = target + 'TS427_' + str(i)
        input = target_indir / file_name
        outpath = target_outdir / file_name
        if input.exists() and not outpath.exists():
            shutil.copy(input, outpath)
    if len(list(target_outdir.glob('*'))) == 0:
        os.removedirs(target_outdir)

../../pdb/alphafold2/T1024
../../pdb/alphafold2/T1025
../../pdb/alphafold2/T1026
../../pdb/alphafold2/T1027
../../pdb/alphafold2/T1028
../../pdb/alphafold2/T1029
../../pdb/alphafold2/T1030
../../pdb/alphafold2/T1031
../../pdb/alphafold2/T1032
../../pdb/alphafold2/T1033
../../pdb/alphafold2/T1034
../../pdb/alphafold2/T1035
../../pdb/alphafold2/T1036s1
../../pdb/alphafold2/T1037
../../pdb/alphafold2/T1038
../../pdb/alphafold2/T1039
../../pdb/alphafold2/T1040
../../pdb/alphafold2/T1041
../../pdb/alphafold2/T1042
../../pdb/alphafold2/T1043
../../pdb/alphafold2/T1045s1
../../pdb/alphafold2/T1045s2
../../pdb/alphafold2/T1046s1
../../pdb/alphafold2/T1046s2
../../pdb/alphafold2/T1047s1
../../pdb/alphafold2/T1047s2
../../pdb/alphafold2/T1048
../../pdb/alphafold2/T1049
../../pdb/alphafold2/T1050
../../pdb/alphafold2/T1052
../../pdb/alphafold2/T1053
../../pdb/alphafold2/T1054
../../pdb/alphafold2/T1055
../../pdb/alphafold2/T1056
../../pdb/alphafold2/T1057
../../pdb/alphafold2/T1058
../../pdb/alph